## CSR performance of US companies 2004 - 2018

### Import libraries

In [1]:
import numpy as np
import pandas as pd

### import the data

In [2]:
df = pd.read_csv("esg.csv")
print(df.shape)
df.head(3)

(23862, 16)


,coname,gvkey,fyear,tdc1,tic,sic_2dgt,annret,roa,lnat,sales_growth,lnrevenue,res_all,ceo_female,csr_tot_net,csr_external_net,csr_internal_net
0,AAR CORP,1004,2000,3910.003,AIR,50,0.036908,0.026403,6.555149,-0.146513,6.774515,0,0.0,NaN,NaN,NaN
1,AAR CORP,1004,2001,2047.286,AIR,50,-0.171749,-0.082989,6.566952,-0.269411,6.461032,0,0.0,NaN,NaN,NaN
2,AAR CORP,1004,2002,2080.489,AIR,50,-0.605518,-0.018074,6.533238,-0.050701,6.409084,0,0.0,NaN,NaN,NaN


### Rename and reorder columns

In [3]:
# change column names 
df.rename(columns=
          {'coname': 'company',
            'fyear':'year',
            'tdc1': 'compensation',
            'tic':'ticker',
            'sic_2dgt':"sic",
            'lnat':'log_assets', 
            'lnrevenue': 'log_sales',
            'annret':'return',
            'res_all':'restatement',
            'csr_tot_net':'csr_total',
            'csr_external_net':'csr_external',
            'csr_internal_net':'csr_internal'}, inplace=True)
# list(df.columns) display new column names

In [4]:
# adjust the order of columns
# assign a new list of column names with the desired order
df = df[['company', 'gvkey', 'year', 'ticker', 'sic', 'log_assets', 'log_sales', 'sales_growth', 'return',
 'roa', 'csr_total', 'csr_external', 'csr_internal', 'restatement', 'ceo_female', 'compensation']]
df.head(3)

,company,gvkey,year,ticker,sic,log_assets,log_sales,sales_growth,return,roa,csr_total,csr_external,csr_internal,restatement,ceo_female,compensation
0,AAR CORP,1004,2000,AIR,50,6.555149,6.774515,-0.146513,0.036908,0.026403,NaN,NaN,NaN,0,0.0,3910.003
1,AAR CORP,1004,2001,AIR,50,6.566952,6.461032,-0.269411,-0.171749,-0.082989,NaN,NaN,NaN,0,0.0,2047.286
2,AAR CORP,1004,2002,AIR,50,6.533238,6.409084,-0.050701,-0.605518,-0.018074,NaN,NaN,NaN,0,0.0,2080.489


### Data type, missing values and descriptive statistics

In [5]:
df.dtypes

company          object
gvkey             int64
year              int64
ticker           object
sic               int64
log_assets      float64
log_sales       float64
sales_growth    float64
return          float64
roa             float64
csr_total       float64
csr_external    float64
csr_internal    float64
restatement       int64
ceo_female      float64
compensation    float64
dtype: object

**Convert strings to integers:** 
- `df['sic'] = df['sic'].astype(int)`
- or `df['sic'] = pd.to_numeric(df['sic'], errors='coerce')`

**Deal with NA's**
- drop rows with at least one missng value: `df_row_any = df.dropna(axis = 0, how = "any")`,  *axis* is used to control whether the dropping happens row-wise or column-wise
- drop rows with all values missing: `df_row_all = df.dropna(axis = 0, how = "all")`
- drop rows for specific variables: `df_row_var = df.dropna(subset=['varname'])`
- drop columns with at least one missng value: `df_col_any = df.dropna(axis = 1, how = "any")`
- replace missing values by 0, `df = df.replace(np.nan, 0, reges=True)`.


In [6]:
df.dropna(axis = 0, how = "any", inplace=True) # dropna() by default does not change the original data, but you can use inplace argument to augment the data
# same as 
# df = df.dropna(axis = 0, how = "any")
print(df.shape)

(19108, 16)


**Descriptive statistics**:
- only one variable: `df.var-name.describe()`
- only numeric columns: `df.describe(include = [np.number])`
- only object columns: `df.describe(include = [object])`
- exclude object columns: `df.describe(exclude = [object])`
- check number of observations: `df.count()`

### Sort data and adjust index

In [7]:
# sort panel data by both company id (gvkey) and year
df = df.sort_values(['gvkey', 'year'])
# set index for all observations in order after sorting, no need to specify variables 
df = df.reset_index(drop=True) # remove the index column
df.iloc[:, 0:3].head(5)

,company,gvkey,year
0,AAR CORP,1004,2003
1,AAR CORP,1004,2004
2,AAR CORP,1004,2005
3,AAR CORP,1004,2006
4,AAR CORP,1004,2007


**Set index**:
- set index by one variable: `df.set_index('varname')`
- set index by two variables: `df.set_index(['var1', 'var2'])`

### Add and drop variables

In [ ]:
# take logarithm
df['log_totcomp'] = np.log(1 + df['compensation'])
df.log_totcomp.describe()

In [ ]:
# create lagged variables
df['lag_log_assets'] = df.groupby('gvkey')['log_assets'].shift(1)
df.lag_log_assets.head(3)
# it is missing value for the first year observation, if needed, you can drop these missing values
# df = df.dropna(subset=['lag_log_assets'])

In [ ]:
# drop columns
df = df.drop(['lag_log_assets'], axis = 1)
# or df = df.drop(columns = ['lag_log_assets'])

# drop columns by index
# df = df.drop(df.iloc[:, -10:], axis=1)

In [8]:
# create categorical variables from numeric variables

ind_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, np.inf]
ind_names = ['Agriculture', 'Construction', 'Manufacturing', 'Transportation', 'Wholesale', 'Retail',
        "Finance", 'Services', 'Public Admin', 'Other']

df['industry'] = pd.cut(df['sic'], ind_bins, labels = ind_names)
df['industry'].value_counts()
# or df['year'].size()
# or df.groupby(['industry]).size()

# count by industry and year
# df[['industry', 'year']].value_counts(ascending=True)

Transportation    6402
Manufacturing     3319
Services          2950
Retail            2275
Construction      1984
Wholesale         1479
Public Admin       450
Agriculture        130
Other               66
Finance             53
Name: industry, dtype: int64

In [ ]:
# Check descriptive statistics by group
# df.groupby(['industry','year'])['log_assets'].describe()

**Delete rows by conditions**

In [9]:
# drop observations by value
df = df.loc[df['year']>2003].reset_index(drop=True) # data points are more stable after 2003.
df.shape

(16792, 17)

In [ ]:
# check unique values of a column
len(df['industry'].unique())
# or df['industry'].nunique()

# check unique values of multiple columns
df[['industry', 'year']].nunique()

# check unique values of the grouping of multiple columns
df[['industry' , 'year']].value_counts()

In [10]:
# drop observations by categories 
(~df['industry'].isin(['Public Admin', 'Agriculture', 'Other','Finance'])).value_counts()

df = df[(~df['industry'].isin(['Public Admin', 'Agriculture', 'Other','Finance']))].reset_index(drop=True)
df['industry'].value_counts()

Transportation    5592
Manufacturing     2902
Services          2610
Retail            1974
Construction      1755
Wholesale         1325
Agriculture          0
Finance              0
Public Admin         0
Other                0
Name: industry, dtype: int64

In [11]:
# drop unused categories
df= df.assign(cats=df['industry'].cat.remove_unused_categories())
df= df.drop(columns='industry')
df=df.rename(columns={'cats': 'industry'})
print(df.shape)
df['industry'].value_counts()

(16158, 17)


Transportation    5592
Manufacturing     2902
Services          2610
Retail            1974
Construction      1755
Wholesale         1325
Name: industry, dtype: int64

In [ ]:
# create dummy variables for each category
dummies = pd.get_dummies(df['industry'])
print(dummies[:10])
df = pd.concat([df, dummies], axis = 1)
print(df.shape)

Create a dummy variable to indicate whether the variable is NA or not:
`df[assets_na] = np.where(df['log_assets'].isna(), 0, 1)`

### Winsorize data

In [ ]:
from scipy.stats.mstats import winsorize
# threshold = 0.01
# df['asset_winsorized'] = winsorize(df['log_assets'].values, limits=threshold)

### Save data

In [ ]:
df.to_csv('esg_cleaned.csv', index=False)